Installation and Dependencies

In [ ]:
# Install MLflow with LLM support
!pip install mlflow[extras]
!pip install transformers torch accelerate
!pip install openai anthropic  # For API-based models

# Optional: Install evaluation dependencies
!pip install rouge-score bert-score sacrebleu
!pip install datasets

In [2]:
!pip install mlflow pyngrok -q

In [ ]:
!pip install -U datasets


In [3]:
import mlflow
import subprocess
from pyngrok import ngrok, conf
import getpass

In [4]:
import os
os.environ["HF_TOKEN"]=""

In [5]:
from pyngrok import ngrok

# Add your token here
ngrok.set_auth_token("")


In [6]:
from datasets import load_dataset
dataset=load_dataset("dair-ai/emotion")
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [7]:
df=dataset['train'].to_pandas()
label_names=dataset['train'].features['label'].names
label_names

df['label_text']=df['label'].apply(lambda x:label_names[x])
df

,text,label,label_text
0,i didnt feel humiliated,0,sadness
1,i can go from feeling so hopeless to so damned...,0,sadness
2,im grabbing a minute to post i feel greedy wrong,3,anger
3,i am ever feeling nostalgic about the fireplac...,2,love
4,i am feeling grouchy,3,anger
...,...,...,...
15995,i just had a very brief time in the beanbag an...,0,sadness
15996,i am now turning and i feel pathetic that i am...,0,sadness
15997,i feel strong and good overall,1,joy
15998,i feel like this was such a rude comment and i...,3,anger


In [8]:
from sklearn.model_selection import train_test_split

#70% of data =Train data, 20% of data=Test data, 10% of data =Validation data

train,test=train_test_split(df,test_size=0.3,stratify=df['label_text'])
test,validation=train_test_split(test,test_size=1/3,stratify=test['label'])

train.shape,test.shape,validation.shape

((11200, 3), (3200, 3), (1600, 3))

In [9]:
from datasets import Dataset,DatasetDict

dataset=DatasetDict({
    'train':Dataset.from_pandas(train,preserve_index=False),
    'test':Dataset.from_pandas(test,preserve_index=False),
    'validation':Dataset.from_pandas(validation,preserve_index=False)
})
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 11200
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 3200
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1600
    })
})

In [10]:
dataset['train'][0],dataset['test'][0],dataset['validation'][0]

({'text': 'i started feeling joyful again i could push those comments out of my heart and live joyfully again',
  'label': 1,
  'label_text': 'joy'},
 {'text': 'im feeling exceptionally brave that day to tell the husband that i need to buy it because i like it and nothing else',
  'label': 1,
  'label_text': 'joy'},
 {'text': 'i feel a bit uncertain really shes a nice girl and good friend material',
  'label': 4,
  'label_text': 'fear'})

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [14]:
def tokenize(batch):
  return tokenizer(batch['text'],padding=True,truncation=True)

In [15]:
encoded=dataset.map(tokenize,batched=True,batch_size=None)
encoded

Map:   0%|          | 0/11200 [00:00<?, ? examples/s]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 11200
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3200
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1600
    })
})

In [16]:
# Define the MLflow tracking URI with SQLite
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

# Start the MLflow server using subprocess
subprocess.Popen(["mlflow", "ui", "--backend-store-uri", MLFLOW_TRACKING_URI, "--port", "5000"])

<Popen: returncode: None args: ['mlflow', 'ui', '--backend-store-uri', 'sqli...>

In [17]:
# Set MLflow tracking URI
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [19]:
# Set or create an experiment
mlflow.set_experiment("llm-fine-tuning-experiment")

2025/06/15 11:45:39 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/06/15 11:45:39 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/06/15 11:45:39 INFO mlflow.tracking.fluent: Experiment with name 'llm-fine-tuning-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/content/mlruns/1', creation_time=1749987939270, experiment_id='1', last_update_time=1749987939270, lifecycle_stage='active', name='llm-fine-tuning-experiment', tags={}>

In [20]:
# Expose the MLflow UI on port 5000
port = 5000
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel "{public_url}" -> "http://127.0.0.1:{port}"')

 * ngrok tunnel "https://bc15-34-168-154-44.ngrok-free.app" -> "http://127.0.0.1:5000"


In [18]:
# Start an MLflow run
with mlflow.start_run():
    # Log parameters and metrics
    mlflow.log_param("learning_rate", 0.01)
    mlflow.log_metric("rmse", 0.25)
    print("Run logged successfully!")

Run logged successfully!


Experiment Tracking for LLM Training

In [21]:
from sklearn.metrics import accuracy_score,f1_score

def compute_metrics(pred):
  labels=pred.label_ids
  preds=pred.predictions.argmax(-1)
  f1=f1_score(labels,preds,average='weighted')
  acc=accuracy_score(labels,preds)
  return {'f1':f1,'accuracy':acc}

In [24]:
from transformers import AutoModel

model=AutoModel.from_pretrained("bert-base-cased")

model.config.id2label,model.config.label2id

label2id={label:i for i ,label in enumerate(label_names)}
id2label={i:label for i ,label in enumerate(label_names)}

label2id,id2label

({'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5},
 {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'})

In [26]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer,AutoConfig
import torch

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config=AutoConfig.from_pretrained("bert-base-cased",id2label=id2label,label2id=label2id)
model=AutoModelForSequenceClassification.from_pretrained("bert-base-cased",config=config)

model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [29]:
import mlflow
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import torch

def fine_tune_llm(train_dataset,eval_dataset):
    with mlflow.start_run(run_name="bert-base-cased-fine-tune") as run:
        # Log hyperparameters
        batch_size=64
        training_args = TrainingArguments(
            output_dir="./results",
              overwrite_output_dir=True,
              num_train_epochs=2,
              learning_rate=2e-5,
              per_device_eval_batch_size=batch_size,
              per_device_train_batch_size=batch_size,
              weight_decay=0.01,
              eval_strategy="epoch",
              disable_tqdm=False,
              report_to='tensorboard',
              logging_strategy="epoch"
        )

        mlflow.log_params({
            "model_name": "bert-base-cased",
            "learning_rate": training_args.learning_rate,
            "batch_size": training_args.per_device_train_batch_size,
            "epochs": training_args.num_train_epochs,
            "warmup_steps": training_args.warmup_steps,
        })



        # Training loop (simplified)
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics,
        )

        # Log training metrics automatically
        trainer.train()

        # Evaluate and log metrics
        eval_results = trainer.evaluate()
        for key, value in eval_results.items():
            mlflow.log_metric(key, value)

        # Log final model
        mlflow.transformers.log_model(
            transformers_model={
                "model": model,
                "tokenizer": tokenizer,
            },
            artifact_path="fine_tuned_model",
            registered_model_name="bert-base-cased"
        )

        return run.info.run_id

In [30]:
fine_tune_llm(encoded['train'],encoded['validation'])

<ipython-input-29-2934095736>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.189600,0.167671,0.929167,0.930000
2,0.101800,0.156455,0.934407,0.933750


2025/06/15 12:12:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Device set to use cuda:0
2025/06/15 12:12:49 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/06/15 12:12:49 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.21.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torchvision==0.21.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/06/15 12:13:04 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.21.0+cu124) 

'b38ef79926b94086993f447aecd70214'

In [32]:
"https://freedium.cfd/https://blog.devgenius.io/mlops-for-llms-with-mlflow-a-complete-guide-550e57243754"

'https://freedium.cfd/https://blog.devgenius.io/mlops-for-llms-with-mlflow-a-complete-guide-550e57243754'

In [68]:
import mlflow.transformers

# Load model from MLflow
model_uri = "runs:/b38ef79926b94086993f447aecd70214/fine_tuned_model"  # or use a registered name: "models:/bert-base-cased/1"
loaded_model = mlflow.transformers.load_model(model_uri)


2025/06/15 12:56:22 INFO mlflow.transformers: 'runs:/b38ef79926b94086993f447aecd70214/fine_tuned_model' resolved as '/content/mlruns/1/b38ef79926b94086993f447aecd70214/artifacts/fine_tuned_model'


2025/06/15 12:56:22 WARNING mlflow.transformers.model_io: Could not specify device parameter for this pipeline type.Falling back to loading the model with the default device.
Device set to use cuda:0


In [69]:
encoded["test"]["text"][:3]

['im feeling exceptionally brave that day to tell the husband that i need to buy it because i like it and nothing else',
 'i were discussing on freedom and economic growth in global civil society i cudnt help but feeling amazed our frens in da philippines dat they r happy maintain basic living condition without rapid development as long as their freedom is not being touched',
 'i feel dissatisfied with the advances we ve made rather than this team stinks']

In [70]:
import mlflow
import mlflow.transformers
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import pipeline

# Sample test data (update if needed)
texts = encoded["train"]["text"][:3]
labels = encoded["train"]["label"][:3]  # Use numeric labels for metrics

# Initialize loaded model as classification pipeline
loaded_model = pipeline("text-classification", model="bert-base-cased")  # Or load from MLflow registry

# Start a new MLflow run for evaluation
with mlflow.start_run(run_name="bert-evaluation"):
    predictions = []
    for text in texts:
        output = loaded_model(text)
        pred_label = int(output[0]["label"].split("_")[-1])  # "LABEL_1" → 1
        predictions.append(pred_label)

    # Calculate metrics
    acc = accuracy_score(labels, predictions)
    prec = precision_score(labels, predictions, average="weighted")
    rec = recall_score(labels, predictions, average="weighted")
    f1 = f1_score(labels, predictions, average="weighted")

    # Log metrics to MLflow
    mlflow.log_metrics({
        "test_accuracy": acc,
        "test_precision": prec,
        "test_recall": rec,
        "test_f1": f1
    })

    print("✅ Evaluation metrics logged to MLflow.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


✅ Evaluation metrics logged to MLflow.


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [96]:
!pkill -f ngrok


In [117]:
import subprocess
import threading

def run_server():
    subprocess.call([
        "mlflow", "models", "serve",
        "-m", "models:/bert-base-cased/2",
        "-p", "5000", "--no-conda"
    ])

# Run in background thread
threading.Thread(target=run_server).start()


In [118]:
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print("🔗 Public Inference Endpoint:", public_url)


🔗 Public Inference Endpoint: NgrokTunnel: "https://1807-34-168-154-44.ngrok-free.app" -> "http://localhost:5000"


In [119]:
import requests
res = requests.get(f"https://ce72-34-168-154-44.ngrok-free.app/ping")
print(res.text)  # should return "OK"


In [120]:
res

<Response [200]>

In [121]:
import requests
import json
import pandas as pd

# Replace this with your actual public ngrok URL
url = "https://ce72-34-168-154-44.ngrok-free.app/invocations"

# Input text(s) under `inputs` key
payload = {
    "inputs": [
        "This is a fantastic product!",
        "I'm really disappointed with this movie."
    ]
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, headers=headers, json=payload)

print("🔍 Status:", response.status_code)
print("📦 Prediction:", response.json())


🔍 Status: 400
📦 Prediction: {'error_code': 'BAD_REQUEST', 'message': 'Encountered an unexpected error while evaluating the model. Verify that the serialized input Dataframe is compatible with the model for inference.', 'stack_trace': 'Traceback (most recent call last):\n  File "/usr/local/lib/python3.11/dist-packages/mlflow/pyfunc/scoring_server/__init__.py", line 367, in invocations\n    raw_predictions = model.predict(data, params=params)\n                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/usr/local/lib/python3.11/dist-packages/mlflow/pyfunc/__init__.py", line 840, in predict\n    return self._predict(data, params)\n           ^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/usr/local/lib/python3.11/dist-packages/mlflow/pyfunc/__init__.py", line 890, in _predict\n    return self._predict_fn(data, params=params)\n           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/usr/local/lib/python3.11/dist-packages/mlflow/transformers/__init__.py", line 1879, in predict\n    return sel

Basic Configuration